# The Great 2011 Tōhoku earthquake and the Fukushima Daiichi Accident: An Analysis

In [38]:
# Hide
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import geopandas as gpd
import folium

In [40]:
#Hide
df = pd.read_csv("/kaggle/input/earthquakes-in-japan/Japan earthquakes 2001 - 2018.csv")

In [56]:
df['id']
df.columns

In [63]:
#Hide
df['popup'] = pd.to_datetime(df['time']).dt.strftime("%Y-%m-%d")+": Magnitude "+df['mag'].astype(str)+": "+df['place'].str.upper()
df.sort_values('mag')

In [71]:
# Loading the background map
eq_map = folium.Map(location=[37.973265, 142.597713], zoom_start=5, tiles='Stamen Terrain', width=1250)
fg = folium.FeatureGroup(name="My Map")

# Adding a circle for each earthquake
for idx, eq in df.query('mag >= 6').iterrows():
    #folium.Circle(location=(eq['latitude'], eq['longitude']), radius=20000 * (eq['mag']-6), color='red', fill=False, alpha = 0.4).add_to(eq_map)
    fg.add_child(folium.CircleMarker(location=[eq['latitude'], 
                                               eq['longitude']], 
                                     radius=10 * (eq['mag']-6), 
                                     color = 'red', 
                                     fill_color='red', 
                                     fill_opacity=0.4, 
                                     popup=eq['popup']))
    
eq_map.add_child(fg)
folium.Circle(location = (37.4218, 141.0337), color = 'yellow', fill_color = 'yellow', radius = 2000, popup="Fukushima Daiichi", fill_opacity=0.4).add_to(eq_map)
eq_map

In [77]:
fig, ax = plt.subplots(figsize=(16,16))
japan_gdf.plot(color =  'lightgray', edgecolor = 'k', ax =ax, zorder = 10, alpha = 0.5)
df.sort_values('mag', inplace=True)
sns.scatterplot(x = df['longitude'], 
                y = df['latitude'], 
                hue = df['mag'],
                s = 2.6**df['mag'],
                #sizes=(minsize, maxsize),
                ax = ax, 
                alpha = 0.8, palette= 'Reds',marker = '.');
plt.xlim(110,170)
plt.ylim(20,60)
plt.show();

In [79]:
freq = (df
         .mag
         .value_counts()
         .sort_index()
         .rename_axis('mag')
         .reset_index(name='counts')
         .assign(freq = lambda x: x.counts/(pd.to_datetime(df['time']).dt.year.max() - \
                                            pd.to_datetime(df['time']).dt.year.min()))
       #  .drop(['counts'], axis =1)
      #   .query('mag <= 6.5')
      .assign(log_freq = lambda x: np.log10(x.freq))
        )
(freq
.plot
.scatter(x = 'mag', y = 'freq', logy=True, grid = True)
)

In [80]:
X = np.array(freq['mag']).reshape(-1,1)
y = np.array(freq['log_freq']).reshape(-1,1)
s = np.array(freq['counts'])

In [81]:
lm = LinearRegression()

lm.fit(X,y,s)

print(f"log10(Frequency) = {lm.coef_[0][0]:.2f}*[Magnitude] + {lm.intercept_[0]:.2f}\n")
print(f"Frequency = 10**({lm.coef_[0][0]:.2f}*[Magnitude] + {lm.intercept_[0]:.2f})")

In [82]:
(freq
.plot
.scatter(x = 'mag', y = 'freq', logy=True, grid = True)
)
plt.plot(np.arange(4.5,9.1,0.1), 
         10**lm.predict(np.arange(4.5,9.1,0.1).reshape(-1,1)), 
         color = 'red', 
         linestyle = "--")